# Finding Detection Mistakes with FiftyOne

Finding mistakes in your annotations can be extremely tedious. The mistakenness feature of FiftyOne can be used to help you find annotation mistakes. Check out [our classification tutorial](https://voxel51.com/docs/fiftyone/tutorials/label_mistakes.html) to see how FiftyOne can help you find and correct label mistakes in your classification datasets.

This recipe is designed to show you how you can use FiftyOne to compute mistakenness on your detection dataset, enabling you to curate higher quality datasets and, ultimately, train better models!

## Overview

In this recipe, we explore how FiftyOne can be used to help you find mistakes in your detection annotations.

Requirements:

- A detection model trained on the same label schema as the annotations you want to analyze
- A FiftyOne Dataset with your annotations and predictions from the model with logits for each detection


We'll cover the following concepts:

-   Computing insights into your detection dataset relating to possible mistakes
-   Visualizing the mistake in the FiftyOne App

## Setup

Your Dataset should have two `Detections` fields, one with your ground truth annotations and one with your model predictions.

In this example, I used the `coco-2017-validation` dataset from the [FiftyOne zoo](https://voxel51.com/docs/fiftyone/user_guide/dataset_creation/zoo.html) and added predictions from the [PyTorch implementation of Faster-RCNN](https://github.com/pytorch/vision/blob/master/torchvision/models/detection/faster_rcnn.py)

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

#dataset = foz.load_zoo_dataset("coco-2017", split="validation")

#add_custom_predictions(dataset)

print(dataset)

NameError: name 'dataset' is not defined

## Find the mistakes

Now we can run a method from the FiftyOne Brain that estimates the mistakenness of the
ground truth detections for which we generated predictions:

In [8]:
import fiftyone.brain as fob

# Compute mistakenness
fob.compute_mistakenness(dataset, model_name, label_field="ground_truth")

Computing mistakenness for 1000 samples...
 100% |███████████████████████████| 1000/1000 [1.3s elapsed, 0s remaining, 808.1 samples/s]         
Mistakenness computation complete


The above method added `mistakenness` field to all samples for which we added
predictions. We can easily sort by likelihood of mistakenness from code:

In [9]:
# Sort by likelihood of mistake (most likely first)
mistake_view = (dataset
    .match_tag("processed")
    .sort_by("mistakenness", reverse=True)
)

# Print some information about the view
print(mistake_view)

Dataset:        cifar10-test
Num samples:    1000
Tags:           ['test', 'processed', 'mistake']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    resnet50:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    mistakenness: fiftyone.core.fields.FloatField
Pipeline stages:
    1. <fiftyone.core.stages.MatchTag object at 0x7f9cb80dbc50>
    2. <fiftyone.core.stages.SortBy object at 0x7f9d4bbd14e0>


In [10]:
# Inspect the first few samples
print(mistake_view.head())

<Sample: {
    'dataset_name': 'cifar10-test',
    'id': '5ef384e36696dbdeabc6a88e',
    'filepath': '/home/voxel51/fiftyone/cifar10/test/data/00107.jpg',
    'tags': BaseList(['test', 'processed']),
    'ground_truth': <Classification: {'label': 'deer'}>,
    'resnet50': <Classification: {
        'label': 'horse',
        'logits': array([-0.83586901, -1.28598607,  1.54965878, -0.49650264, -0.40103185,
               -0.18043809, -1.0332154 ,  5.05314684, -1.21831954, -1.15143788]),
    }>,
    'mistakenness': 1.0,
}>
<Sample: {
    'dataset_name': 'cifar10-test',
    'id': '5ef384e36696dbdeabc6a86f',
    'filepath': '/home/voxel51/fiftyone/cifar10/test/data/00076.jpg',
    'tags': BaseList(['test', 'processed']),
    'ground_truth': <Classification: {'label': 'bird'}>,
    'resnet50': <Classification: {
        'label': 'deer',
        'logits': array([-0.72157425, -0.94043797, -0.32308894, -0.19049911,  4.82478857,
               -0.35608411, -0.35027471, -0.25426134, -0.77823019, 

Let's use the App to visually inspect the results:

In [11]:
# Launch the FiftyOne App
session = fo.launch_app()

# Open your dataset in the App
session.dataset = dataset

App launched


![dataset](images/label_mistakes_1.png)

In [12]:
# Show only the samples that were processed
session.view = dataset.match_tag("processed")

![processed](images/label_mistakes_2.png)

In [13]:
# Show only the samples for which we added label mistakes
session.view = dataset.match_tag("mistake")

![mistake](images/label_mistakes_3.png)

In [14]:
# Show the samples we processed in rank order by the mistakenness
session.view = mistake_view

![mistake-view](images/label_mistakes_4.png)